I can't get geo location for ALL records in the PPP dataset (it's cost prohibitive!).
But I can tally the amounts for by state and figure out per capita costs. . . 



In [1]:
import pandas as pd

In [ ]:
import glob
workDir = '/home/alp/Google Drive/Python/analysis/PPP_analysis/data/'

file_names = sorted(glob.glob(workDir+'public_up_to*.csv'))
firstFile = 'public_150k_plus_220930.csv'

colList = ['BorrowerName','BorrowerAddress', 'BorrowerCity','BorrowerState','BorrowerZip','CurrentApprovalAmount','JobsReported','NAICSCode','Race','Ethnicity','BusinessType','ForgivenessAmount']

df = pd.read_csv(workDir+firstFile, usecols = colList)

for file in file_names:
    oldDF = df
    print(file)
    if firstFile not in file:
        newDF = pd.read_csv(file, usecols = colList)
        df = pd.concat([oldDF,newDF])

df.info()        

In [5]:
# add the FIPS information to PPP
# this file of population estimates has FIPS info we can add to the PPP data
pop_df = pd.read_excel('data/PopulationEstimates.xlsx',sheet_name='StateTotals')

pop_df.rename(columns={'Federal Information Processing Standards (FIPS) Code':'FIPS',
                        'Population 1990': 'pop1990', 'Population 2000':'pop2000',
                        'Population 2010':'pop2010','Population 2020':'pop2020',
                        'Population 2021':'pop2021','Area name':'County'},inplace=True)

pop_df.head(5)


,FIPS,State,pop1990,pop2000,pop2010,pop2020,pop2021
0,NaN,State,NaN,NaN,NaN,NaN,NaN
1,NaN,US,NaN,NaN,NaN,NaN,NaN
2,NaN,AL,4040389.0,4447207.0,4779736.0,5024279.0,NaN
3,NaN,AK,550043.0,626933.0,710231.0,733391.0,NaN
4,NaN,AZ,3665339.0,5130247.0,6392017.0,7151502.0,NaN


In [6]:
map1 = df.groupby(['BorrowerState'])['ForgivenessAmount'].sum()
# .count() creates a series, where the index is the FIPS number and the value column is "LoanNumber"
map1 = map1.reset_index()
# .reset_index() converts the series to a df, with two columns: 'FIPS" and 'LoanNumber'
map1.rename(columns={'ForgivenessAmount':'TotalCost'},inplace=True) 
# this step isn't needed, but we're not looking at a LoanNumber, but at the number of loans

In [2]:
4/24

0.16666666666666666

In [8]:
map1a = pd.merge(left=map1,right=pop_df,left_on='BorrowerState',right_on='State',how='left')

In [16]:
# create a new column that divides the N loans by the population in 2020
map1a['CostPerCap'] = round((map1a['TotalCost'] / map1a['pop2020']))


In [13]:
map1a.to_json('TotalStateCosts.json')

In [2]:
workDir = '/home/alp/Google Drive/Python/analysis/PPP_analysis/'
fileN = 'TotalStateCosts.json'

map1a = pd.read_json(f"{workDir}{fileN}")

In [6]:
# get state boundary data
import geopandas as gpd
states = gpd.read_file("/home/alp/Google Drive/Python/GeoShapeFiles/cb_2021_us_state_5m.zip")
states.rename(columns={'STUSPS':'state'},inplace = True)
states = states[['state','geometry']]

In [20]:
import folium

m1= folium.Map(location=[40,-90],
                tiles='OpenStreetMap',zoom_start = 4)

# this map uses clusters to keep from getting overwhemed by the number of points
# it doesn't work very well, becuase the clusters are too large

title = 'Cost of PPP loans per person by state'

cp = folium.Choropleth(
    geo_data = states,
    name = 'choropleth',
    data = map1a, 
    columns = ['State','CostPerCap'],
    key_on = 'feature.properties.state',
    bins = 5,
    fill_color='OrRd',
    fill_opacity=0.8,
    line_opacity=0.2,
    nan_fill_color = 'grey',
    legend_name=title
    ).add_to(m1)

mapdata = map1a.set_index('State')

for row in cp.geojson.data['features']:
    try:
        row['properties']['CostPerCap'] = str(mapdata.loc[row['properties']['state'],'CostPerCap'])
    except KeyError:
        row['properties']['CostPerCap'] = 'No loans'

folium.GeoJsonTooltip(['state','CostPerCap'],aliases=['State:','Total PPP Cost PP:']).add_to(cp.geojson)

folium.LayerControl().add_to(m1)

m1